<a href="https://colab.research.google.com/github/malibio/project-1-deep-learning-image-classification-via-cnn/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Global imports

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import time
import numpy as np
import tarfile
import pickle
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input


Load CIFAR-10 data

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))


for layer in base_model.layers:
    layer.trainable = False


Normalize the Images

In [5]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


Convert Labels to One-Hot Encoding

In [6]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Split Validation Set

In [7]:
validation_split = 0.2
num_train_samples = int((1 - validation_split) * x_train.shape[0])
x_train_final = x_train[:num_train_samples]
y_train_final = y_train[:num_train_samples]
x_val = x_train[num_train_samples:]
y_val = y_train[num_train_samples:]

 Resize Images to 224x224

In [9]:
IMG_SIZE = 224


 Image Preprocessing Function


In [10]:


def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label

Build Efficient Datasets

In [11]:
#Train
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_final, y_train_final))
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
#validation
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
#test
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)



batch_size = 32

In [12]:
vgg16_base = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
vgg16_base.trainable = False

Build Our Own Model on Top

In [13]:
vgg16_base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
vgg16_base.trainable = False
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = vgg16_base(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,140,042 (80.64 MB)

 Trainable params: 6,425,354 (24.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Compile the Model

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Train the Model

In [15]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_vgg16_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
]

In [ ]:
start_time = time.time()
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks
)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")


Epoch 1/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 288s 219ms/step - accuracy: 0.0966 - loss: 2.4718 - val_accuracy: 0.1025 - val_loss: 2.3027
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 330s 235ms/step - accuracy: 0.0975 - loss: 2.3028 - val_accuracy: 0.0952 - val_loss: 2.3027
Epoch 3/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 322s 235ms/step - accuracy: 0.1004 - loss: 2.3027 - val_accuracy: 0.0980 - val_loss: 2.3027
Epoch 4/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 322s 235ms/step - accuracy: 0.1037 - loss: 2.3026 - val_accuracy: 0.0952 - val_loss: 2.3026
Epoch 5/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 322s 235ms/step - accuracy: 0.0987 - loss: 2.3028 - val_accuracy: 0.0952 - val_loss: 2.3027
Epoch 6/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 264s 211ms/step - accuracy: 0.0991 - loss: 2.3027 - val_accuracy: 0.1022 - val_loss: 2.3026
Epoch 7/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 352s 235ms/step - accuracy: 0.0965 - loss: 2.3027 - val_accuracy: 0.0952 - val_loss: 2.3027
Epoch 8/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 292s 211ms/step - ac

Evaluate the Model

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_acc}")

Make Predictions

In [ ]:
for images, labels in test_dataset.take(1):
    predictions = model.predict(images[:3])
    print(predictions)